In [1]:
from context_analyser import *

In [2]:
CA = ContextAnalyser(False)
CA.read_sentences('files/zdania_trening.txt')
CA.read_contexts('files/konteksty_trening.txt')
CA.read_sentences('files/zdania.txt')
CA.read_contexts('files/konteksty.txt')

In [3]:
X, Y = CA.getXY()

In [5]:
Y.shape

(201, 8)

In [6]:
CA.get_context(Y[10].argmax())

'astronomia'

In [7]:
from sklearn.neural_network import MLPClassifier
from math import floor

In [8]:
def result(X, y, clf, CA, print_res=True):
    score = 0.0
    for i in range(len(y)):
        pred_idx = clf.predict_proba([X[i]])[0].argmax()
        org_idx = y[i].argmax()
        pred_res = CA.get_context(pred_idx)
        org_res = CA.get_context(org_idx)
        score += (org_res == pred_res)
        if print_res:
            print("pred:{}, oryg:{}".format(pred_res, org_res))
    print(score / len(y))

def train_test_split(X, Y, train_size=0.9, shuffle_test=False, scale=True):
    size = len(X)
    delim = floor(size*train_size)+1
    X_train = X[:delim]
    y_train = Y[:delim]
    X_test = X[delim:]
    y_test = Y[delim:]
    train_order = np.arange(delim)
    test_order = np.arange(len(y_test))
    np.random.shuffle(train_order)
    if shuffle_test:
        np.random.shuffle(test_order)
    if scale:
        X_train = np.arctan(X_train)
        X_test = np.arctan(X_test)
    return X_train[train_order], y_train[train_order], X_test[test_order], y_test[test_order]

In [9]:
class Scaler():
    def __init__(self):
        self.min = 0
        self.max = 0
        
    def fit(self, X):
        self.min = X.min()
        self.max = X.max()

    def transform(self, X):
        return (X - self.min) / (self.max - self.min)

    def inverse_transform(self, X):
        return X * (self.max - self.min) + self.min

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [10]:
scaler = Scaler()
X_scaled = scaler.fit_transform(X)

In [11]:
X_train, y_train, X_test, y_test = train_test_split(X_scaled, Y, 0.9, True)

In [16]:
clf = MLPClassifier(solver='adam', alpha=1e-2,
                    hidden_layer_sizes=(2,4), max_iter=10000).fit(X_train, y_train)

result(X_train, y_train, clf, CA, False)
result(X_test, y_test, clf, CA, False)

1.0
0.55


In [17]:
result(X_test, y_test, clf, CA, True)

pred:zwierzęta, oryg:zwierzęta
pred:zwierzęta, oryg:sport
pred:zwierzęta, oryg:zwierzęta
pred:owoc, oryg:owoc
pred:owoc, oryg:owoc
pred:zwierzęta, oryg:astronomia
pred:sport, oryg:sport
pred:alkohol, oryg:sport
pred:zwierzęta, oryg:zwierzęta
pred:astronomia, oryg:astronomia
pred:sport, oryg:owoc
pred:astronomia, oryg:astronomia
pred:owoc, oryg:owoc
pred:alkohol, oryg:alkohol
pred:zwierzęta, oryg:historia
pred:zwierzęta, oryg:wypadek
pred:alkohol, oryg:historia
pred:sport, oryg:owoc
pred:sport, oryg:owoc
pred:owoc, oryg:owoc
0.55


In [93]:
result(X_test, y_test, clf, CA, False)

0.05263157894736842


In [85]:
A = np.array([10,12,3, 150])


In [25]:
B = np.arange(4)
np.random.shuffle(B)
A[B]

array([  3,  10,  12, 150])

array([  3,  12, 150,  10])